In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import binom_test
from tabulate import tabulate
sns.set(font_scale=1.8)

In [2]:
result_dir = 'results'

In [3]:
def create_plots(file):
    df = pd.read_csv(os.path.join(result_dir, file))
    if 'DL' in file:
        df = df.multiply(100)


    def calculate_p_values(df):
        data = df.values.copy()/100
        # Assuming equal number of trials for each subject and finger pair
        num_trials = 100

        # Calculate p-values for each classification accuracy
        p_values = np.zeros_like(data)
        for i in range(data.shape[0]):
            for j in range(data.shape[1]):
                success_count = int(data[i, j] * num_trials)
                p_values[i, j] = binom_test(success_count, num_trials, p=0.5)

        # print("P-values:")
        # print(p_values)

        # Find elements with p-values less than 0.05
        significant_elements = p_values < 0.05

        # print("\nSignificantly above chance level (p<0.05):")
        # print(significant_elements)
        return p_values

    p_values = calculate_p_values(df)


    # Create heatmap using seaborn
    subjects = ['S1', 'S2', 'S3', 'S4', 'S5']
    plt.figure(figsize=(12, 6))
    heatmap = sns.heatmap(df, yticklabels = subjects, annot=True, cmap="YlGnBu")
    plt.xticks(rotation=45)

    labels = heatmap.get_xticklabels()
    heatmap.set_xticklabels(labels, ha='right')

    plt.title("Binary Finger Classification")

    plt.tight_layout()
    plt.savefig(f'plots/{file.split(".")[0]}_heatmap.png')


        
    # Create box plot using seaborn
    plt.figure(figsize=(12, 6))
    plt.title("Classifier Results Mean(Std) for each Subject", y=1.15)
    ax = sns.boxplot(data=df.T)
    ax.set_ylim(40, 100)
    plt.xlabel("Subjects")
    plt.ylabel("Percentage")
    ax.set_xticklabels(subjects)

    mean_fingers = df.mean(axis=1)
    std_fingers = df.std(axis=1)

    # Add mean and standard deviation values as annotations
    for i, (mean, std) in enumerate(zip(mean_fingers, std_fingers)):
        plt.text(i - 0.3, 102, f"{mean:.1f}({std:.1f})", fontsize=20)
    
    plt.tight_layout()
    plt.savefig(f'plots/{file.split(".")[0]}_box.png')


In [4]:

def create_tables(file):
    df = pd.read_csv(os.path.join(result_dir, file))
    if 'DL' in file:
        df = df.multiply(100)

    def calculate_p_values(df):
        data = df.values.copy()/100
        # Assuming equal number of trials for each subject and finger pair
        num_trials = 100

        # Calculate p-values for each classification accuracy
        p_values = np.zeros_like(data)
        for i in range(data.shape[0]):
            for j in range(data.shape[1]):
                success_count = int(data[i, j] * num_trials)
                p_values[i, j] = binom_test(success_count, num_trials, p=0.5)

        return p_values

    p_values = calculate_p_values(df)

    subjects = ['S1', 'S2', 'S3', 'S4', 'S5']

    # Create an annotation array with values and p-values
    rounded_data = np.round(df.values).astype(int)
    significant_elements = p_values < 0.05
    annot_array = np.empty(df.shape, dtype=object)

    for i in range(df.shape[0]):
        for j in range(df.shape[1]):
            if significant_elements[i, j]:

                if p_values[i, j] < 0.001:
                    annot_array[i, j] = str(rounded_data[i, j]) + "(\\textbf{<0.001})"
      
                else:
                    annot_array[i, j] = f"{rounded_data[i, j]} (\\textbf{{{p_values[i, j]:.3f}}})"
            else:
                annot_array[i, j] = f"{rounded_data[i, j]} ({p_values[i, j]:.3f})"

    # Transpose the annotation array
    #annot_array_T = annot_array.T

    # Create a LaTeX table using tabulate
    headers = [column.replace('_', '/') for column in list(df.columns)]
    table_data = [headers]

    for i, row in enumerate(annot_array):
        table_data.append([subjects[i]] + list(row))

    # Print the LaTeX table
    latex_table = tabulate(table_data, headers='firstrow', tablefmt='latex_booktabs')
    latex_table = latex_table.replace('\\textbackslash{}textbf\{', '\\textbf{').replace('\}', '}')
    print(latex_table)


    # Save the LaTeX table to a text file
    with open(f'tables/{file.split(".")[0]}_latex_table.txt', 'w') as f:
        f.write(latex_table)


In [5]:
for file in os.listdir(result_dir):
    create_tables(file)

\begin{tabular}{lllllllllll}
\toprule
    & little/index        & middle/index        & middle/little       & ring/index          & ring/little         & ring/middle         & thumb/index         & thumb/little        & thumb/middle        & thumb/ring          \\
\midrule
 S1 & 61 (\textbf{0.035}) & 66 (\textbf{0.002}) & 59 (0.089)          & 86(\textbf{\ensuremath{<}0.001}) & 85(\textbf{\ensuremath{<}0.001}) & 75(\textbf{\ensuremath{<}0.001}) & 67(\textbf{\ensuremath{<}0.001}) & 63 (\textbf{0.012}) & 71(\textbf{\ensuremath{<}0.001}) & 88(\textbf{\ensuremath{<}0.001}) \\
 S2 & 62 (\textbf{0.035}) & 62 (\textbf{0.021}) & 64 (\textbf{0.012}) & 60 (0.057)          & 65 (\textbf{0.004}) & 65 (\textbf{0.007}) & 66 (\textbf{0.004}) & 62 (\textbf{0.021}) & 57 (0.193)          & 68(\textbf{\ensuremath{<}0.001}) \\
 S3 & 65 (\textbf{0.004}) & 66 (\textbf{0.004}) & 65 (\textbf{0.004}) & 72(\textbf{\ensuremath{<}0.001}) & 74(\textbf{\ensuremath{<}0.001}) & 80(\textbf{\ensuremath{<}0.001}) & 70(\